In [113]:
import cloudscraper
from bs4 import BeautifulSoup
import math
import pprint


# Takes a public Discogs store inventory and returns pricing information on other listings on the market.


scraper = cloudscraper.create_scraper(browser={'browser': 'firefox','platform': 'windows','mobile': False})


# Helper Functions

def get_inventory(username):

    URL = "https://www.discogs.com/seller/{0}/profile".format(username)
    pages = count_pages(URL)
    
    return parse_list(URL, pages)
    
def parse_list(URL, pages): # Takes URL of a store inventory, returns the releases as a list.
    
    new_list = []

    for page in range(1, pages + 1):
        new_URL = URL + "?&limit=250&sort=price&sort_order=asc&page={0}".format(page)
        html = scraper.get(new_URL).content
        soup = BeautifulSoup(html, 'html.parser')

        list_items = soup.find(id="pjax_container").find("tbody").find_all("tr")
        for item in list_items:
            release = item.find("td", class_="item_description")
            title = release.find("strong").text.strip()
            item_id = release.find("a", class_="item_release_link")["href"].split("-")[0].strip("/release/")

            new_list_item = (title, item_id)
            new_list.append(new_list_item)
    
    return new_list

def count_pages(URL): # Takes URL for a Discogs store, returns the number of pages.
    
    html = scraper.get(URL).content
    soup = BeautifulSoup(html, 'html.parser')
    
    inventory_size = int(soup.find(id="page_content").find("li", class_="first").find("h2").text.strip().strip("For Sale"))
    pages = math.ceil(inventory_size/250)
            
    return pages

def get_listings(username, item_id): # Given username and item_id, scrapes marketplace listings for prices.
    
    URL = "https://www.discogs.com/sell/release/{0}?ships_from=United+States&sort=price%2Casc".format(item_id)
    html = scraper.get(URL).content
    soup = BeautifulSoup(html, 'html.parser')
    
    print(URL)
    
    count = 0
    listings = soup.find("table", class_="mpitems").find_all("tr", class_="shortcut_navigable")
    for listing in listings:
        count += 1
        if is_user(username, listing):
            print("{0} (You - {1})".format(get_price(listing), count))
            # break
        else:
            print(get_price(listing))
    
    total = (soup.find("strong", class_="pagination_total").text.split(" of "))[-1]
    print("Total: {0}".format(total))
    
    return

def is_user(username, listing): # Checks if a marketplace listing matches the provided username.
    
    seller = listing.find("div", class_="seller_block").find("a").text    
    return seller == username

def get_price(listing): # Gets price provided a listing.

    try:
        item_condition = listing.find("p", class_="item_condition").text
        formatted_condition = format_condition(item_condition)
        
        if listing.find(string="New seller"):
            return "{0} {1} (New)".format(listing.find("span", class_="converted_price").text.strip(), formatted_condition)
        else:
            return "{0} {1}".format(listing.find("span", class_="converted_price").text.strip(), formatted_condition)
    except AttributeError:
        return "n/a"
    
def format_condition(item_condition): # Formats the item condition to (Media/Sleeve).
    
    # print(item_condition)
    
    media = (item_condition.split("("))[1].split(")")[0]
    try:
        sleeve = (item_condition.split("("))[2].split(")")[0]
    except IndexError:
        return "({0})".format(media.split(" or")[0])

    media = media.split(" or")[0]
    sleeve = sleeve.split(" or")[0]
    
    return "({0}/{1})".format(media, sleeve)


# Main

print("Enter username: ")
user1 = input()
print("")

try:
    inventory = get_inventory(user1)

    count = 0
    for release in inventory:
        count += 1
        print("({0})".format(count))

        print(release[0])
        get_listings(user1,release[1])
        print("\n")
        
except AttributeError:
    print("Can't find user.")

# get_listings(user1, 26058199)

Enter username: 
curefortheitch

(1)
Ariana Grande - Positions (LP, Album, Ltd, Glo)
https://www.discogs.com/sell/release/18263449?ships_from=United+States&sort=price%2Casc
$20.99 total (M/NM)
$26.99 total (NM/VG+)
$30.45 total (M/NM)
$31.00 total (M/M) (New)
$35.00 total (NM/NM) (You - 5)
$40.00 total (M/NM)
$41.00 total (M/M)
$39.99 total (M/NM)
Total: 8    


(2)
Madeintyo - Sincerely, Tokyo (LP, Album, Red)
https://www.discogs.com/sell/release/13188544?ships_from=United+States&sort=price%2Casc
$24.99 total (M/VG+)
$27.99 total (VG+/VG+)
$31.00 total (M/NM)
$35.00 total (NM/NM) (You - 4)
$41.48 total (M/M)
$44.00 total (NM/NM)
$46.00 total (M/VG)
Total: 7    


(3)
Kygo - Cloud Nine (2xLP, Album, Whi)
https://www.discogs.com/sell/release/8512721?ships_from=United+States&sort=price%2Casc
$33.75 total (NM/VG+)
$38.99 total (NM/NM) (You - 2)
Total: 2    


(4)
SPVCXXGHXZTPVRRP* - Mysterious Phonk: The Chronicles Of SPVCXXGHXZTPVRRP (2xLP, Album, Ltd, Num, RE, Red)
https://www.discogs.c

https://www.discogs.com/sell/release/21604510?ships_from=United+States&sort=price%2Casc
$78.99 total (M/M) (You - 1)
$82.69 total (M/M)
Total: 2    


(26)
Tink (7) - Hopeless Romantic (LP, Album, Pin)
https://www.discogs.com/sell/release/15734458?ships_from=United+States&sort=price%2Casc
$70.50 total (NM/VG+)
$84.99 total (M/VG+) (You - 2)
Total: 2    


(27)
Flume - Hi, This Is Flume (LP, Mixtape, RP, Gre)
https://www.discogs.com/sell/release/15457869?ships_from=United+States&sort=price%2Casc
$84.99 total (M/VG+) (You - 1)
$98.08 total (M/NM)
$98.00 total (M/M)
$102.99 total (M/M) (New)
$105.00 total (M/M)
$123.99 total (M/M)
$149.99 total (M/M)
$203.99 total (NM/NM)
$227.99 total (NM/NM)
$251.50 total (M/M)
$254.00 total (M/M)
Total: 11    


(28)
Linkin Park - A Thousand Suns (2xLP, Album, Gat)
https://www.discogs.com/sell/release/2578691?ships_from=United+States&sort=price%2Casc
$91.99 total (M/M)
$89.99 total (NM/NM)
$92.99 total (M/NM) (You - 3)
$93.00 total (M/M)
$95.99 total (

https://www.discogs.com/sell/release/8967794?ships_from=United+States&sort=price%2Casc
$88.75 total (NM/NM) (New)
$119.00 total (NM/VG+)
$133.00 total (NM/VG+)
$155.99 total (VG+/VG+)
$156.00 total (M/M)
$205.00 total (M/NM) (You - 6)
$205.00 total (M/VG+)
Total: 7    


(52)
ミラクルミュージカル - Hawaii: Part II (LP, Album, Ltd, RP, 180)
https://www.discogs.com/sell/release/25973644?ships_from=United+States&sort=price%2Casc
$145.00 total (M/M) (New)
$163.00 total (M/NM)
$205.00 total (M/NM) (You - 3)
$213.71 total (M/M)
$420.00 total (M/M)
Total: 5    


(53)
Joyce Wrice - Overgrown  (LP, Album, Ltd, Whi)
https://www.discogs.com/sell/release/19319761?ships_from=United+States&sort=price%2Casc
$87.00 total (NM/NM) (New)
$154.98 total (M/NM)
$205.00 total (NM/NM) (You - 3)
$205.97 total (NM)
$283.98 total (M/M)
$285.00 total (M/M) (New)
$291.09 total (M/M)
$500.00 total (M)
Total: 8    


(54)
Beast Coast - Escape From New York (2xLP, Album, Ltd, Blu)
https://www.discogs.com/sell/release/13672908

https://www.discogs.com/sell/release/16223662?ships_from=United+States&sort=price%2Casc
$201.25 total (VG+/NM) (New)
$305.00 total (NM/VG+)
$334.99 total (VG+/VG) (You - 3)
Total: 3    


(78)
$uicideboy$ - I Want To Die In New Orleans (LP, Album, Bla)
https://www.discogs.com/sell/release/12730110?ships_from=United+States&sort=price%2Casc
$329.00 total (VG+/NM) (New)
$334.99 total (NM/NM) (You - 2)
Total: 2    


(79)
Marilyn Manson - Antichrist Superstar (2xLP, Album, Ltd)
https://www.discogs.com/sell/release/3222700?ships_from=United+States&sort=price%2Casc
$104.00 total (NM)
$354.99 total (VG/VG) (You - 2)
$501.50 total (NM/NM)
Total: 3    


(80)
Mariah Carey - Caution (LP, Album, Ltd, Hot)
https://www.discogs.com/sell/release/13049389?ships_from=United+States&sort=price%2Casc
$255.00 total (NM/VG+)
$355.00 total (M/VG+) (You - 2)
$436.00 total (M/NM)
$431.00 total (M/M)
$505.89 total (M/M)
$512.00 total (M/NM)
Total: 6    


(81)
Gunna - Wunna (2xLP, Album, Dlx)
https://www.discog